# 边缘检测
 用于检出图像边缘
 边缘是像素值发生剧烈变化的地方,是图像的显著特征之一,在图像特征提取,对象检测,模式识别方面有重要作用
 ![img](常用的边缘检测算子.png)

In [1]:
import cv2
import numpy as np

## 索贝尔算子(sobel)
一阶微分算子，X、Y方向边缘检测
缺点 误差较大,不够敏感

https://blog.csdn.net/great_yzl/article/details/119709699
![img](sobel.png)

ex007.jpg
<img src="../data/ex007.jpg" width=200 height=200>

In [2]:
# 使用sobel算子 
#  Sobel(src, ddepth(位深), dx, dy[, dst[, ksize[, scale[, delta[, borderType]]]]]) -> dst
# x 和 y 一般要分开计算(dx和dy同时使用 效果会比较差) 然后计算整体梯度 
img = cv2.imread('../data/ex007.jpg',0) # 读灰度图(彩色图也可以)


# 计算 水平梯度 x方向 结果只保留垂直的线条 位深对精度影响很大
gx = cv2.Sobel(img,cv2.CV_64F,dx=1,dy=0,ksize=5) # 默认是ksize=3

# 计算 垂直梯度 y方向 结果只保留水平的线条
gy = cv2.Sobel(img,cv2.CV_64F,dx=0,dy=1,ksize=5)

# 把gx 和 gy 合并 求整体梯度 3种不同的求和方式
gxy1 = cv2.add(gx,gy)
gxy2 = cv2.addWeighted(gx,0.5,gy,0.5,0)
gxy3 = np.uint8(np.sqrt(np.power(gx,2)+np.power(gy,2))) 


temp = np.hstack((img,gx,gy,gxy1,gxy2,gxy3))
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
cv2.imshow('ex',temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./sobel_gx_gy.jpg',temp)

True

./sobel_gx_gy.jpg
<img src="./sobel_gx_gy.jpg"  >

## 沙尔算子(Scharr)
### 改进sobel算子精度较差的缺陷(核大小固定为3*3,不可以选)
### 更擅长寻找细小的边缘,一般情况下sobel也够用了
<img src="./scharr.png">

In [3]:
# Scharr(src, ddepth, dx, dy[, dst[, scale[, delta[, borderType]]]]) -> dst
# x 和 y 一般要分开计算(dx和dy同时使用 效果会比较差) 然后计算整体梯度 
img = cv2.imread('../data/linna.jpg') 


# 计算 水平梯度 x方向 结果只保留垂直的线条 位深对精度影响很大
gx = cv2.Scharr(img,cv2.CV_64F,dx=1,dy=0)

# 计算 垂直梯度 y方向 结果只保留水平的线条
gy = cv2.Scharr(img,cv2.CV_64F,dx=0,dy=1)

# 把gx 和 gy 合并 求整体梯度 3种不同的求和方式
gxy1 = cv2.add(gx,gy)
gxy2 = cv2.addWeighted(gx,0.5,gy,0.5,0)
gxy3 = np.uint8(np.sqrt(np.power(gx,2)+np.power(gy,2))) 


temp = np.hstack((img,gx,gy,gxy1,gxy2,gxy3))
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
cv2.imshow('ex',temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./scharr_gx_gy.jpg',temp)

True

<img src="./scharr_gx_gy.jpg"  >

## 拉普拉斯算子
sobel算子是模拟求一阶导,导数取得极值的地方可能就是边缘,如果在一阶导的基础上继续求导,二阶导数为0的点可能就是边缘

要注意 二阶导数为0的点也可能不是极值点

sobel算子和scharr算子一般先算一个水平梯度，再算一个垂直方向梯度，然后把两个结果按照0.5的权重进行图像融合以得到完整的边界。

但Laplacian算子则不同，它本身就是一个二阶的算子，它的运算规则就是在水平方向运算两次，垂直方向运算两次，两个结果相叠加替换中心点（锚点）的像素值（灰度值）
<img src="./拉普拉斯算子.png"  >
可以看出 拉普拉斯算子对类似椒盐噪声特别敏感,因此使用前通常需要进行降噪处理(中值滤波)

In [4]:
# Laplacian(src, ddepth[, dst[, ksize[, scale[, delta[, borderType]]]]]) -> dst
img =cv2.imread('./img_salt.png',0)
img_me = cv2.medianBlur(img,7)
img_la = cv2.Laplacian(img_me,cv2.CV_64F)

temp = np.uint8(np.hstack((img,img_me,img_la)))
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
cv2.imshow('ex',temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./img_la.jpg',temp)

True

![img](./img_la.jpg)

In [5]:
img =cv2.imread('../data/ex007.jpg',0) # 这张图片没有什么噪声
# img_me = cv2.medianBlur(img,7)
img_la = cv2.Laplacian(img,cv2.CV_64F)
print(img_la.dtype)
temp = np.hstack((img,img_la))
print(temp.dtype)
cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
cv2.imshow('ex',temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./img_la2.jpg',temp)

float64
float64


True

![img](./img_la2.jpg)

## canny算子边缘检测
https://zhuanlan.zhihu.com/p/99959996

 (1)   高斯模糊
 
（2） 使用Sobel算子计算像素梯度大小和方向

    g =(gx^2+gy^2)^0.5
    θ = arctan(gy/gx)
    
<img src="./20210118145737745.png"  >

<img src="./v2-d0712c8a0b6aaea4d84fe45f454d0109_r.jpg"  >

（3） 非极大值像素梯度抑制


（4） 阈值滞后处理

（5） 孤立弱边缘抑制

<img src="./canny边缘检测流程.png"  >

In [18]:
# 使用 Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) -> edges
img =cv2.imread('../data/linna.jpg',0) # 使用灰度图

img_gaussianblur= cv2.GaussianBlur(img,(3,3),100)

edge1 = cv2.Canny(img_gaussianblur,100,200)
edge2 = cv2.Canny(img_gaussianblur,50,200)

cv2.namedWindow('ex',cv2.WINDOW_NORMAL)
cv2.imshow('ex',np.hstack((img,img_gaussianblur,edge1,edge2)))
cv2.waitKey(0)
cv2.destroyAllWindows()

